In [ ]:
# Julia


## Introduction

In labor economics an important question is what determines the wage of workers. This is a causal question, but we could begin to investigate from a predictive perspective.

In the following wage example, $Y$ is the hourly wage of a worker and $X$ is a vector of worker's characteristics, e.g., education, experience, gender.
Two main questions here are:    

* How to use job-relevant characteristics, such as education and experience, to best predict wages?

* What is the difference in predicted wages between men and women with the same job-relevant characteristics?

In this lab, we focus on the prediction question first.

## Data

The data set we consider is from the March Supplement of the U.S. Current Population Survey, year 2015. We select white non-hispanic individuals, aged 25 to 64 years, and working more than 35 hours per week during at least 50 weeks of the year. We exclude self-employed workers; individuals living in group quarters; individuals in the military, agricultural or private household sectors; individuals with inconsistent reports on earnings and employment status; individuals with allocated or missing information in any of the variables used in the analysis; and individuals with hourly wage below $3$.

The variable of interest $Y$ is the hourly wage rate constructed as the ratio of the annual earnings to the total number of hours worked, which is constructed in turn as the product of number of weeks worked and the usual number of hours worked per week. In our analysis, we also focus on single (never married) workers. The final sample is of size $n = 5150$.

## Data Analysis

In [2]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Dates")
Pkg.add("Plots")
using CSV
using DataFrames
using Dates
using Plots

    Updating registry at `C:\Users\Kenia\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Kenia\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Kenia\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Kenia\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Kenia\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Kenia\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Kenia\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Kenia\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Kenia\.julia\environments\v1.7\Manifest.toml`


In [25]:
#Reading the CSV file into a DataFrame
#We have to set the category type for some variable
data = CSV.File("C:\Users\Kenia\Documents\Draft_kenia\data\wage2015_subsample_inference-Copy1.Rdata"; types = Dict("occ" => String,"occ2"=> String,"ind"=>String,"ind2"=>String)) |> DataFrame
println("Number of Rows : ", size(data)[1],"\n","Number of Columns : ", size(data)[2],) #rows

LoadError: syntax: invalid escape sequence

In [4]:
[eltype(col) for col = eachcol(data)]

LoadError: UndefVarError: data not defined

In [5]:
first(data,10)

LoadError: UndefVarError: data not defined

In [6]:
describe(data)

LoadError: UndefVarError: data not defined

In [7]:
n = size(data)[1]
z = select(data, Not([:rownames, :lwage, :wage]))
p = size(z)[2] 

println("Number of observations : ", n, "\n","Number of raw regressors:", p )

LoadError: UndefVarError: data not defined

In [8]:
z_subset = select(data, ["lwage","sex","shs","hsg","scl","clg","ad","mw","so","we","ne","exp1"])
describe(z_subset, :mean)

LoadError: UndefVarError: data not defined

## Prediction Question

Now, we will construct a prediction rule for hourly wage $Y$ , which depends linearly on job-relevant characteristics  $X$:

$$Y = \beta' X + \epsilon $$
 
Our goals are

* Predict wages using various characteristics of workers.

* Assess the predictive performance using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample $MSE$ and $R^2$.

We employ two different specifications for prediction:

- **Basic Model**: $X$ consists of a set of raw regressors (e.g. gender, experience, education indicators, occupation and industry indicators, regional indicators).

- **Flexible Model**: $X$ consists of all raw regressors from the basic model plus occupation and industry indicators, transformations (e.g.,$exp2$ and $exp3$) and additional two-way interactions of polynomial in experience with other regressors. An example of a regressor created through a two-way interaction is experience times the indicator of having a college degree.

Using the **Flexible Model**, enables us to approximate the real relationship by a more complex regression model and therefore to reduce the bias. The **Flexible Model** increases the range of potential shapes of the estimated regression function. In general, flexible models often deliver good prediction accuracy but give models which are harder to interpret.

Now, let us fit both models to our data by running ordinary least squares (ols):

In [9]:

Pkg.add("Plots")
Pkg.add("Lathe")
Pkg.add("GLM")
Pkg.add("StatsPlots")
Pkg.add("MLBase")
Pkg.add("StatsModels")
Pkg.add("Combinatorics")
# Load the installed packages
using DataFrames
using CSV
using Plots
using Lathe
using GLM
using Statistics
using StatsPlots
using MLBase
using StatsModels
using Combinatorics

   Resolving package versions...
  No Changes to `C:\Users\Kenia\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Kenia\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Kenia\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Kenia\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Kenia\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Kenia\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Kenia\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Kenia\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
    Updating `C:\Users\Kenia\.julia\environments\v1.7\Project.toml`
  [f0e99cf1] + MLBase v0.8.0
  No Changes to `C:\Users\Kenia\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
    Updating `C:\Users\Kenia\.julia\environments\v1.7\Project

In [10]:
#basic model
basic  = @formula(lwage ~ (sex + exp1 + shs + hsg + mw + so + we + occ2+ ind2))
basic_results  = lm(basic, data)

LoadError: UndefVarError: data not defined

In [11]:
# couples variables combinations
combinations_upto(x, n) = Iterators.flatten(combinations(x, i) for i in 1:n)
# combinations without same couple
expand_exp(args, deg::ConstantTerm) =
    tuple(((&)(terms...) for terms in combinations_upto(args, deg.n))...)

StatsModels.apply_schema(t::FunctionTerm{typeof(^)}, sch::StatsModels.Schema, ctx::Type) =
    apply_schema.(expand_exp(t.args_parsed...), Ref(sch), ctx)


In [14]:
#flexible model
flex = @formula(lwage ~ sex + (exp1+exp2+exp3+exp4 +shs+hsg+occ2+ind2+mw+so+we)^2)
flexi = @formula(sex ~ (exp1+exp2+exp3+exp4 +shs+hsg+occ2+ind2+mw+so+we)^2)
regflex = lm(flex, data=data)
regflex = lm(flexi, data=data)

LoadError: UndefVarError: data not defined

In [ ]:
## Lasso

In [26]:
Pkg.add("Lasso")
using Lasso

   Resolving package versions...
  No Changes to `C:\Users\Carol\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Carol\.julia\environments\v1.7\Manifest.toml`


In [ ]:
flex = @formula(lwage ~ sex + (exp1+exp2+exp3+exp4 +shs+hsg+occ2+ind2+mw+so+we)^2)
flexi = @formula(sex ~ (exp1+exp2+exp3+exp4 +shs+hsg+occ2+ind2+mw+so+we)^2)
lasso_model = fit(LassoModel, flex, data)
lasso_model = fit(LassoModel, flexi, data)

